# HashiCorp Vault Demo for KMIP Secrets Engine and Oracle Database Enterprise TDE

Note: This notebook is still not fully functioning as we are missing the ARM64 pluging for this to work. Work in progress.

The OASIS Key Management Interoperability Protocol (KMIP) standard is a widely adopted protocol for handling cryptographic workloads and secrets management for enterprise infrastructure such as databases, network storage, and virtual/physical servers.  When an organization has services and applications that need to perform cryptographic operations (e.g.: transparent database encryption (TDE), full disk encryption, etc.), it often delegates the key management task to an external provider via KMIP protocol. 

This demo demonstrates HashiCorp Vault's KMIP Secrets Engine to allow Vault to function as a KMIP server for clients that that retrieve cryptographic keys for encrypting data via the KMIP protocol.

Ref: https://developer.hashicorp.com/vault/docs/enterprise/pkcs11-provider/oracle-tde

## Setup of the Demo

This setup is tested on MacOS and is meant to simulate a distributed setup.  The components used in this demo are:
- Vault Enterprise installed on docker (to simulate an external Vault)
- Oracle Database Enterprise installed on docker (to simulate an external Oracle DB that requires TDE encryption.  i.e. encryption at rest.)
- You have the Vault CLI installed

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables.

Note: You will need Vault to be installed with an ADP KMIP license add-on.  Also note that a Premium license is required if you wish the KMIP listener to scale on the Vault cluster.  For Standard or Plus license, the KMIP listener is only on the leader node.

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

We will require an Oracle account to access the Oracle Database Enterprise trial.

- Go to https://edelivery.oracle.com and create an account (Click on "Create Account").  Note: Register with a valid email and details otherwise the account might not be allowed to download.  Enter your account details below.
- Go to https://container-registry.oracle.com and login.
- Search for "oracle database enterprise" and click on the Oracle Database Enterprise Edition repository link.
- Accept the License Agreement for Oracle Database Enterprise Edition.
- See the tags of the container version that you wish to use and update the docker commands below.

In [ ]:
# Important! Update your oracle account details here.
export ORACLE_REG_EMAIL=
export ORACLE_REG_PASSWORD=

# Use the Oracle DB Enterprise docker image for linux/arm64
# Using 19c as that is the only build with arm64 support
export ORACLEDB_TAG=19.19.0.0

# Log into the Oracle docker registry
echo "$ORACLE_REG_PASSWORD" | docker login container-registry.oracle.com -u $ORACLE_REG_EMAIL --password-stdin
# Pull the MySQL Enterprise docker image
docker pull container-registry.oracle.com/database/enterprise:$ORACLEDB_TAG

# Setting up HashiCorp Vault

In [ ]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
# Expose both the Vault API and the KMIP ports to the host machine.
export VAULT_PORT=8200
export VAULT_KMIP_PORT=5696
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-p ${VAULT_KMIP_PORT}:${VAULT_KMIP_PORT} \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

# Vault KMIP Secrets Engine
This section demonstrates the KMIP secrets engine and how it functions as a KMIP server with Oracle Database TDE.

Ref:
- https://developer.hashicorp.com/vault/docs/secrets/kmip

## Step 1 - Enable Vault KMIP Engine

In [ ]:
# Enable KMIP secrets engine at the default mount path
#vault secrets disable kmip
vault secrets enable kmip

In [ ]:
# Start KMIP Server and set client TLS certificate TTL to 365 days

# Option 1 - Elliptic Curve (EC) is the default
vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT default_tls_client_ttl=365d
# Option 2 - RSA
#vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT tls_ca_key_type="rsa" tls_ca_key_bits=2048 default_tls_client_ttl=365d

# Check the KMIP configuration
echo
vault read kmip/config

## Step 2 - Creating the KMIP certificates for Oracle Database TDE

In [ ]:
# Create a folder for the KMIP certificates
mkdir -p root/etc

In [ ]:
# On MacOS, the casing doesn't matter.
vault read -format=json kmip/ca | jq -r .data.ca_pem > root/etc/ca.pem

echo "CA certification:"
cat root/etc/ca.pem

## Step 2a - Create KMIP Scope in Vault

The KMIP secrets engine uses the concept of scopes to partition KMIP managed object storage into multiple named buckets. Within a scope, roles can be created which dictate the set of allowed operations that the particular role can perform. TLS client certificates can be generated for a role, which services and applications can then use when sending KMIP requests against Vault's KMIP secret engine.

In order to generate client certificates for KMIP clients to interact with Vault's KMIP server, we must first create a scope and role and specify the desired set of allowed operations for it.

In [ ]:
# Settings for the KMIP scope and role to be created
export KMIP_SVC_NAME=my-service
export KMIP_ROLE_NAME=admin
# Create a scope
vault write -f kmip/scope/$KMIP_SVC_NAME

# List scopes and verify the KMIP scope has been created
echo
vault list kmip/scope

In [ ]:
# Create the KMIP role (note the TTL if not set, it uses the default Vault token/lease TTL of 768hrs)
vault write kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME operation_all=true tls_client_ttl=365d

# List roles and verify the KMIP role has been created
echo
vault list kmip/scope/$KMIP_SVC_NAME/role

In [ ]:
# Optional - View the role details
vault read kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME

In [ ]:
# Generate the required certificates
vault write -format=json \
  kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME/credential/generate \
  format=pem > credential.json

# Save the certificate and private key
jq -r .data.certificate < credential.json > root/etc/cert.pem
jq -r .data.private_key < credential.json > root/etc/key.pem

# Remove the temp file
rm credential.json

## Step 3 - Configuring the PKCS11 Config File for TDE with Vault as the KMIP server

In [ ]:
# Get the docker IP address of the Vault server
export VAULT_IP=$(docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' vault-enterprise)
echo "Vault IP Address is: $VAULT_IP"
echo "Vault KMIP Port is: $VAULT_KMIP_PORT"

# Create the PKCS11 Config File.  Specifies the KMIP server for Oracle DB
# For this demo, we will be using the same Vault server as the standby.
echo
tee root/etc/vault-pkcs11.hcl <<EOF
slot {
  server = "$VAULT_IP:$VAULT_KMIP_PORT"
  tls_cert_path = "/etc/cert.pem"
  ca_path = "/etc/ca.pem"
  scope = "my-service"
}
EOF

In [ ]:
# Download Vault PKCS#11 Provider
# https://releases.hashicorp.com/vault-pkcs11-provider
# Take note that PKCS#11 requires GLIC to run. Do take note of the required GLIC version. 
#wget -P root/etc https://releases.hashicorp.com/vault-pkcs11-provider/0.2.0/vault-pkcs11-provider_0.2.0_linux-el7_amd64.zip

# !!This is the problem, we need an arm plugin for our Oracle ARM based container
curl -o root/etc/vault-pkcs11-provider_0.2.0_linux-el7_amd64.zip \
  https://releases.hashicorp.com/vault-pkcs11-provider/0.2.0/vault-pkcs11-provider_0.2.0_linux-el7_amd64.zip
unzip -d root/etc root/etc/vault-pkcs11-provider_0.2.0_linux-el7_amd64.zip


In [ ]:
# Run Oracle Database Enterprise in docker.
# Expose both the Oracle DB port to the host machine.
export ORACLEDB_PORT=1521
export ORACLEDB_TAG=19.19.0.0
export ORACLE_PASSWORD=MyPassw0rd

# Start the Oracle Enterprise container and bind the KMIP certificate files, PKCS#11 Config file, and PKCS#11 Binary to the proper paths
echo "Oracle Database Container Version Used: $ORACLEDB_TAG"
docker run -d --rm --name oracledb-enterprise \
--mount type=bind,src=./root/etc/ca.pem,dst=/etc/ca.pem \
--mount type=bind,src=./root/etc/cert.pem,dst=/etc/cert.pem \
--mount type=bind,src=./root/etc/vault-pkcs11.hcl,dst=/etc/vault-pkcs11.hcl \
--mount type=bind,src=./root/etc/libvault-pkcs11.so,dst=/opt/oracle/extapi/64/hsm/libvault-pkcs11.so \
-e ORACLE_PWD=$ORACLE_PASSWORD \
-p ${ORACLEDB_PORT}:${ORACLEDB_PORT} container-registry.oracle.com/database/enterprise:$ORACLEDB_TAG


#-e VAULT_LOG_FILE='/opt/oracle/vault.log' \


In [ ]:
# Run Oracle Database Enterprise in docker.
# Expose both the Oracle DB port to the host machine.
# export ORACLEDB_PORT=1521
# export ORACLEDB_TAG=19.19.0.0

# echo "Oracle Database Container Version Used: $ORACLEDB_TAG"
# docker run -d --rm --name oracledb-enterprise \
# -e VAULT_LOG_FILE='/opt/oracle/vault.log' \
# -p ${ORACLEDB_PORT}:${ORACLEDB_PORT} container-registry.oracle.com/database/enterprise:$ORACLEDB_TAG


In [ ]:
# Verify that the Oracle DB docker container is running
docker ps

In [ ]:
# Copy over all KMIP ca, KMIP cert, PKCS#11 Config file and PKCS#11 Binary to Oracle
# docker cp root/etc/ca.pem oracledb-enterprise:/etc/ca.pem
# docker cp root/etc/cert.pem oracledb-enterprise:/etc/cert.pem
# docker cp root/etc/vault-pkcs11.hcl oracledb-enterprise:/etc/vault-pkcs11.hcl
# oracle mkdir -p /opt/oracle/extapi/64/hsm
# docker cp root/etc/libvault-pkcs11.so oracledb-enterprise:/opt/oracle/extapi/64/hsm/libvault-pkcs11.so

#THIS IS FOR TRYING TO DEBUG UNABLE TO FIND PROVIDER ERROR
# Use privilege user to set the correct ownership for the copied file. 
# docker exec -u -0 -it oracledb-enterprise chown oracle:oinstall /etc/ca.pem
# docker exec -u -0 -it oracledb-enterprise chown oracle:oinstall /etc/cert.pem
# docker exec -u -0 -it oracledb-enterprise chown oracle:oinstall /etc/vault-pkcs11.hcl
# docker exec -u -0 -it oracledb-enterprise chown oracle:oinstall -R /opt/oracle/extapi
# docker exec -u -0 -it oracledb-enterprise chmod 755 /opt/oracle/extapi/64/hsm/libvault-pkcs11.so

In [ ]:
# Check and ensure oracledb is fully started. Wait for the completion to be 100%
# Stop this task when you see "DATABASE IS READY TO USE"
docker logs --follow --tail 20 oracledb-enterprise

## Step 4 - Oracle DB TDE Configuration Steps

In [ ]:
# Change the root password to "password"
# export ORACLE_PASSWORD=password
# oracle ./setPassword.sh $ORACLE_PASSWORD

In [ ]:
# set oracle alias to make it easier to execute oracle commands to the oracle db docker container
alias oracle="docker exec -i oracledb-enterprise"

In [ ]:
# Create a new common user (denoted by prefix c##).  Drop if it exists.
# Ref: https://oracle-base.com/articles/12c/multitenant-manage-users-and-privileges-for-cdb-and-pdb-12cr1
export ORACLE_TDE_ADMIN=tdeadmin
oracle sqlplus sys/$ORACLE_PASSWORD@ORCLCDB as sysdba <<EOF
DROP USER c##$ORACLE_TDE_ADMIN;
CREATE USER c##$ORACLE_TDE_ADMIN IDENTIFIED BY $ORACLE_PASSWORD;
GRANT CREATE SESSION TO c##$ORACLE_TDE_ADMIN;
GRANT SYSDBA TO c##$ORACLE_TDE_ADMIN CONTAINER=ALL;
GRANT ALTER SYSTEM TO c##$ORACLE_TDE_ADMIN;
exit
EOF
#GRANT CONNECT, RESOURCE, DBA TO $ORACLE_TDE_ADMIN;
#ALTER SESSION SET "_ORACLE_SCRIPT"=TRUE;  


In [ ]:
# Set the wallet root
#oracle sqlplus sys/$ORACLE_PASSWORD@ORCLCDB as sysdba <<EOF
oracle sqlplus c##$ORACLE_TDE_ADMIN/$ORACLE_PASSWORD@ORCLCDB as sysdba <<EOF
alter system set wallet_root='/opt/oracle/admin/ORCLCDB/wallet' scope=spfile;
exit
EOF

In [ ]:
# Restart Oracle by shutting down first
oracle ./shutDown.sh normal

In [ ]:
# Start up Oracle
oracle ./startUp.sh

In [ ]:
# Set the TDE Configuration
oracle sqlplus c##$ORACLE_TDE_ADMIN/$ORACLE_PASSWORD@ORCLCDB as sysdba <<EOF
alter system set TDE_CONFIGURATION="KEYSTORE_CONFIGURATION=HSM" SCOPE=both;
exit
EOF

In [ ]:
#Check and ensure configuration is correct
oracle sqlplus c##$ORACLE_TDE_ADMIN/$ORACLE_PASSWORD@ORCLCDB as sysdba <<EOF
SELECT name, value from V\$PARAMETER WHERE NAME IN ('wallet_root','tde_configuration');
exit
EOF

In [ ]:
# Open the HSM Wallet. 
#This is the part where if you check the logs, it shows unable to find PKCS#11 library.
oracle sqlplus c##$ORACLE_TDE_ADMIN/$ORACLE_PASSWORD@ORCLCDB as sysdba <<EOF
ADMINISTER KEY MANAGEMENT SET KEYSTORE OPEN IDENTIFIED BY "1234" CONTAINER = ALL;
exit
EOF

In [ ]:
# Create the TDE master key the HSM Wallet. 
oracle sqlplus c##$ORACLE_TDE_ADMIN/$ORACLE_PASSWORD@ORCLCDB as sysdba <<EOF
ADMINISTER KEY MANAGEMENT SET KEYSTORE OPEN IDENTIFIED BY "1234" CONTAINER = ALL;
exit
EOF

## Step 5 - Testing Oracle TDE functionality

# Cleanup

In [ ]:
# Cleanup

# Disable KMIP secrets engine
vault secrets disable kmip

# Remove KMIP certificate demo files
rm -rf root

# Stop Vault container
docker stop vault-enterprise

# Stop Oracle container
docker stop oracledb-enterprise